### 1. Setting up and Gathering Data

In [3]:
!pip install -U pip 
!pip install labelme tensorflow opencv-python matplotlib albumentations

Above installations:
- labelme: GUI to draw image annotations
- tensorflow: deep learning library to train neural net
- opencv: image/video/I/O and CV ops (resize, blur, edges, etc)
- matplotlib: plotting library for charts and visualizations
- albumentations: fast image augmentations for training

### 1.2 Collect Images With OpenCV

In [4]:
import os 
import time
import uuid
import cv2

Libraries above:
- os: file system & OS stuff like paths, env vars, etc..
- time: timestamps, delays, timing
- uuid: generate unique ID's for file names
- cv2: read/write images, resize/rotate, draw, video capture


In [5]:
IMAGE_PATH = os.path.join('data','image')
number_images = 30